# مثال استفاده از ابزار AutoGen


## وارد کردن بسته‌های مورد نیاز


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## ایجاد کلاینت

در این نمونه، از [مدل‌های GitHub](https://aka.ms/ai-agents-beginners/github-models) برای دسترسی به LLM استفاده خواهیم کرد.

`model` به صورت `gpt-4o-mini` تعریف شده است. سعی کنید مدل را به مدل دیگری که در بازار مدل‌های GitHub موجود است تغییر دهید تا نتایج متفاوتی را مشاهده کنید.

به عنوان یک آزمایش سریع، فقط یک درخواست ساده اجرا می‌کنیم - `پایتخت فرانسه کجاست`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## تعریف توابع

در این مثال، ما به عامل دسترسی به ابزاری می‌دهیم که یک تابع با لیستی از مقاصد تعطیلات موجود و وضعیت دسترسی آن‌ها است.

می‌توانید تصور کنید که این یک سناریو است که در آن یک آژانس مسافرتی ممکن است به یک پایگاه داده سفر دسترسی داشته باشد، برای مثال.

همان‌طور که این نمونه را مرور می‌کنید، می‌توانید آزادانه توابع و ابزارهای جدیدی تعریف کنید که عامل بتواند آن‌ها را فراخوانی کند.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## تعریف ابزار تابع  
برای اینکه عامل بتواند از `vacation_destinations` به عنوان یک `FunctionTool` استفاده کند، باید آن را به عنوان یک ابزار تعریف کنیم.  

همچنین توضیحی درباره ابزار ارائه خواهیم داد که به عامل کمک می‌کند تا تشخیص دهد این ابزار در ارتباط با وظیفه‌ای که کاربر درخواست کرده است، برای چه کاری استفاده می‌شود.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## تعریف عامل

اکنون می‌توانیم عامل را در کد زیر ایجاد کنیم. ما `system_message` را تعریف می‌کنیم تا به عامل دستورالعمل‌هایی بدهیم که چگونه به کاربران در یافتن مقاصد تعطیلات کمک کند.

همچنین پارامتر `reflect_on_tool_use` را روی true تنظیم می‌کنیم. این امکان را فراهم می‌کند که LLM پاسخ ابزار را گرفته و آن را با استفاده از زبان طبیعی ارسال کند.

می‌توانید این پارامتر را روی false تنظیم کنید تا تفاوت را مشاهده کنید.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## اجرای عامل

اکنون می‌توانیم عامل را با پیام اولیه کاربر که درخواست سفر به توکیو دارد اجرا کنیم.

شما می‌توانید مقصد این شهر را تغییر دهید تا ببینید عامل چگونه به موجود بودن شهر پاسخ می‌دهد.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما برای دقت تلاش می‌کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌هایی باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه انسانی حرفه‌ای استفاده کنید. ما هیچ مسئولیتی در قبال سوءتفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
